# Basil Docking V0.1 - Data Manipulation and Collection
## Purpose

__Target Audience__<br>
Undergraduate chemistry/biochemistry students and, in general, people that have little to no knowledge of protein-ligand docking and would like to understand the general process of docking a ligand to a protein receptor.

__Brief Overview__<br>
Molecular docking is a computational method used to predict where molecules are able to bind to a protein receptor and what interactions exist between the molecule (from now on, refered to as "ligand") and the receptor. It is a popular technique utilized in drug discovery and design, as when creating new drugs and testing existing drugs aginst new receptors, it is useful to determine the likelihood of binding prior to screening as it can be used to eliminate molecules that are unlikely to bind to the receptor. This significantly reduces the potential cost and time needed to test the efficacy of a set of possible ligands. <br>

The general steps to perform molecular docking, assuming the ligand and receptor are ready to be docked, include the generation of potential ligand binding poses and the scoring of each generated pose (which predicts how strongly the ligand binds to the receptor, with a more negative score corresponding to a stronger bond). To dock a ligand to a protein, (insert text).<br>

This notebook series encompasses 
1. The preparation needed prior to docking (protein and ligand sanitation, ensuring files are in readable formats, and finding possible binding pockets)
2. The process of docking ligand/s to a protein receptor using two docking engines (VINA and SMINA) and visualizing/analyzing the outputs
3. __Further data collection and manipulation__
4. Utilizing machine learning to determine key residues (on the protein) and functional groups (on the ligand) responsible for protein-ligand binding

__Stepwise summary for this notebook (docking data manipulation and collection, notebook 3 out of 4)__
- Create derivatives for ligands by substituting/modifying functional groups on a canonical ligand
- Dock derivative/s to receptor
- Collect and store data including but not limited to the score, interaction type, distance between interacting atoms from the ligand and protein, and functional group involved in interaction


## Table of Libraries Used
### Operations, variable creation, and variable manipulation

| Module (Submodule/s) | Abbreviation| Role | Citation |
| :--- | :--- | :--- | :--- |
| numpy | np | perform mathematical operations and fix NaN values in dataframe outputs | Harris, C.R., Millman, K.J., van der Walt, S.J. et al. Array programming with NumPy. Nature 585, 357–362 (2020). DOI: 10.1038/s41586-020-2649-2. (Publisher link). |
| pandas | pd | organize data in an easy-to-read format and allow for the exporting of data as a .csv file | The pandas development team. (2024). pandas-dev/pandas: Pandas (v2.2.3). Zenodo. https://doi.org/10.5281/zenodo.13819579 |
| re |n/a| regular expression; find and pull specific strings of characters depending on need, allow for easy naming and variable creation | Van Rossum, G. (2020). The Python Library Reference, release 3.8.2. Python Software Foundation. |
| os | n/a| allow for interaction with computer operating system, including the reading and writing of files |  Van Rossum, G. (2020). The Python Library Reference, release 3.8.2. Python Software Foundation. |
| sys |n/a| manipulate python runtime environment |  Van Rossum, G. (2020). The Python Library Reference, release 3.8.2. Python Software Foundation.|
| glob |n/a| pull files of interest, specifically for blind docking |  Van Rossum, G. (2020). The Python Library Reference, release 3.8.2. Python Software Foundation. |
| warnings | n/a | filter warnings | Van Rossum, G. (2020). The Python Library Reference, release 3.8.2. Python Software Foundation. |

### Protein and Ligand Preparation
| Module (Submodule/s)| Abbreviation | Role | Citation |
| :--- | :--- | :--- | :--- |
| open babel (pybel)| n/a | prepare ligand derivatives for docking |  O'Boyle, N.M., Banck, M., James, C.A. et al. Open Babel: An open chemical toolbox. J Cheminform 3, 33 (2011). https://doi.org/10.1186/1758-2946-3-33.|
| rdkit (Chem)| n/a | ligand creation and sanitation |  RDKit: Open-source cheminformatics; http://www.rdkit.org |

### Visualization
| Module (Submodule/s)| Abbreviation | Role | Citation |
| :--- | :--- | :--- | :--- |
| rdkit.Chem (AllChem, Draw)| n/a | derivative visualization |  RDKit: Open-source cheminformatics; http://www.rdkit.org |
| py3Dmol | n/a | apoprotein and protein complex visualization |  Keshavan Seshadri, Peng Liu, and David Ryan Koes. Journal of Chemical Education 2020 97 (10), 3872-3876. https://doi.org/10.1021/acs.jchemed.0c00579. |

### Docking
| Module (Submodule/s)| Abbreviation | Role | Citation |
| :--- | :--- | :--- | :--- |
| vina | n/a | ligand-protein docking |  Eberhardt, J., Santos-Martins, D., Tillack, A.F., Forli, S. (2021). AutoDock Vina 1.2.0: New Docking Methods, Expanded Force Field, and Python Bindings. Journal of Chemical Information and Modeling. |
| --- | --- | --- | Trott, O., & Olson, A. J. (2010). AutoDock Vina: improving the speed and accuracy of docking with a new scoring function, efficient optimization, and multithreading. Journal of computational chemistry, 31(2), 455-461. |
| smina | n/a | ligand-protein docking |  Koes, D. R., Baumgartner, M. P., & Camacho, C. J. (2013). Lessons learned in empirical scoring with smina from the CSAR 2011 benchmarking exercise. Journal of chemical information and modeling, 53(8), 1893–1904. https://doi.org/10.1021/ci300604z |
| fpocket | n/a | find possible binding pockets in protein receptors | Le Guilloux, V., Schmidtke, P. & Tuffery, P. Fpocket: An open source platform for ligand pocket detection. BMC Bioinformatics 10, 168 (2009). https://doi.org/10.1186/1471-2105-10-168. |
|pdbqt_to_sdf | n/a | create sdf files from pdbqt files created from docking with vina | Ruiz-Moreno A.J. Jupyter Dock: Molecular Docking integrated in Jupyter Notebooks. https://doi.org/10.5281/zenodo.5514956 |

### Data analysis
| Module (Submodule/s)| Abbreviation | Role | Citation |
| :--- | :--- | :--- | :--- |
| MDAnalysis | mda | allow for the selection of atoms for separating protein from ligands and ligands from each other | R. J. Gowers, M. Linke, J. Barnoud, T. J. E. Reddy, M. N. Melo, S. L. Seyler, D. L. Dotson, J. Domanski, S. Buchoux, I. M. Kenney, and O. Beckstein. MDAnalysis: A Python package for the rapid analysis of molecular dynamics simulations. In S. Benthall and S. Rostrup, editors, Proceedings of the 15th Python in Science Conference, pages 98-105, Austin, TX, 2016. SciPy, doi:10.25080/majora-629e541a-00e. |
| --- | --- | --- | N. Michaud-Agrawal, E. J. Denning, T. B. Woolf, and O. Beckstein. MDAnalysis: A Toolkit for the Analysis of Molecular Dynamics Simulations. J. Comput. Chem. 32 (2011), 2319-2327, doi:10.1002/jcc.21787. PMCID:PMC3144279. |
| rdkit (Chem)| n/a | reorder/retrieve derivative atoms, retrieve information from derivative sdf files for visualization and comparison  |  RDKit: Open-source cheminformatics; http://www.rdkit.org |
| prolif (Complex3D)| plf | calculate, record, and view protein-ligand interactions|  chemosim-lab/ProLIF: v0.3.3 - 2021-06-11.https://doi.org/10.5281/zenodo.4386984. |

### UI
| Module (Submodule/s)| Abbreviation | Role | Citation |
| :--- | :--- | :--- | :--- |
| IPython (ipywidgets)| n/a | allow for widgets to be implemented | Fernando Pérez, Brian E. Granger, IPython: A System for Interactive Scientific Computing, Computing in Science and Engineering, vol. 9, no. 3, pp. 21-29, May/June 2007, doi:10.1109/MCSE.2007.53. URL: https://ipython.org |
| ipywidgets (Layout, Label, Dropdown, Box)| widgets | create dropdowns for docking engine, pocket number, ligand, and pose selection | Fernando Pérez, Brian E. Granger, IPython: A System for Interactive Scientific Computing, Computing in Science and Engineering, vol. 9, no. 3, pp. 21-29, May/June 2007, doi:10.1109/MCSE.2007.53. URL: https://ipython.org |


For using this notebook, certain libraries are required in order for analysis to perform as planned. You can either use a conda library (provided as a yml file) or install all required libraries using pip install. Only run the cells below if you will not use a conda library to install required libraries, and only use them as needed. If you are using a conda library, start at the coding cell that imports the libraries.

In [ ]:
# create cell to import all libraries via pip install
! pip install numpy

In [ ]:
! pip install pandas

In [ ]:
! pip install ipywidgets

In [ ]:
! pip install mdanalysis

In [ ]:
! pip install openbabel

In [ ]:
! pip install rdkit

In [ ]:
! pip install py3dmol

In [ ]:
! pip install vina

In [ ]:
! pip install smina

In [ ]:
! pip install prolif

In [ ]:
! pip install scipy

In [ ]:
import numpy as np
import pandas as pd
import scipy
from scipy.sparse import csr_matrix
import re
import sys, os
import glob
sys.path.insert(1, 'utilities/')
from utils import pdbqt_to_sdf
import ipywidgets as widgets
from ipywidgets import Layout, Label, Dropdown, Box, SelectMultiple

from openbabel import pybel
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

import prolif as plf
from prolif.plotting.complex3d import Complex3D
import MDAnalysis as mda 
from MDAnalysis.coordinates import PDB

sys.path.insert(1, 'utilities/ligandsplitter/ligandsplitter')
from ligandanalysis import get_vars, group_idxes_from_mol

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

## Import Values from docking-and-prelim-analysis

Prior to docking, the data obtained from the previous notebook needs to be imported in order to be used. The glob library gets the pdb file containing the receptor, and the ligand information is obtained from the ligand_information.csv file created at the end of the previous notebook. The protein pocket information generated from fpocket is imported from the prot_pockets.csv file.

In [ ]:
# get needed values 
prot_pockets = pd.read_csv('data/protein_pockets.csv',index_col=[0])
ligand_information = pd.read_csv('data/ligand_information.csv')

find_pdb = os.path.join('data', 'PDB_files', '*.ent')
prot_file = glob.glob(find_pdb)[0]
prot_file_split = prot_file.split('/')[-1]
pdb_id = prot_file_split[3:7]

In [ ]:
ligs = []
filenames = []
filenames_H = []
filenames_pdbqt = []
for r in ligand_information.index:
    ligs.append(ligand_information["ligs"][r])
    filenames.append(ligand_information["filenames"][r])
    filenames_H.append(ligand_information["filenames_H"][r])
    filenames_pdbqt.append(ligand_information["filenames_pdbqt"][r])

## Data manipulation and collection

### Analyze functional groups in each ligand and create derivatives

To create modified ligands to dock against the receptor, the functional groups of the original ligand/s are identified using substructure matching of SMILES and SMARTS strings pertaining to different functional groups. The names of each functional group present in the original ligand are saved along with the indexes of the atoms in the ligand making up the functional group. The result is a list of ones and zeros called `all_func_groups`, where a "1" corresponds to a functional group being present and a "0" corresponds to a functional group being absent. If the functional groups of multiple ligands are being idendified, `all_func_groups` will be a nested list containing the results for each ligand.

Derivatives of each original ligand will be created by replacing one of its identified functional groups with a different functional group. To reduce the number of small ligands created due to fragmentation, ligands with a SMILES string containing less than 25 characters will be ignored and will not be used for docking. If there are any inconsistancies within the derived ligand (i.e. atoms marked as aromatic that shouldn't be, atoms not following traditional binding rules) the derived ligand will be ignored and will not be used for docking.

In [ ]:
derivative_mols = []
derivative_smile = []

groups_dict = {
    'ester': ['C(=O)-[NH]-C', 'C[C](=O)[O][C](=O)C'],
    'ether': ['C-S-C'],
    'hydroxy': ['C(=O)[OH]', 'C(=O)[H]', 'C(=O)-Cl'],
    'carbox_acid': ['C(=O)-[H]', 'C(=O)-Cl'],
    'aldehyde': ['C(=O)[OH]', 'C(=O)-Cl', 'C-[OH]', 'C-S-[H]', 'C(=[NH])[H]'],
    'anhydr': ['C(=O)-[NH]-C','C(=O)OC', 'C(=O)-[NH]-C'],
    'amine': ['CC', 'C-[N+](=O)-[O-]'],
    'amide': ['C(=O)[OH]', 'C(=O)[H]'],
    'amide_2': ['C(=O)OC'],
    'nitro': ['C-[NH2]'],
    'f_hal': ['Cl', 'Br', 'I'],
    'cl_hal': ['F', 'Br', 'I'],
    'br_hal': ['F', 'Cl', 'I'],
    'i_hal': ['F', 'Cl', 'Br'],
    'alkyne_term': ['C#N'], 
    'phenyl' : ['Cc1ccc([OH])cc1','Cc1ccc(OC)cc1', 'Cc1ccc(C)cc1', 'Cc1ccccc1'],
    'benzyl': ['CCc1ccc([OH])cc1','CCc1ccc(OC)cc1', 'CCc1ccc(C)cc1', 'CCc1ccccc1'],
    'pyrrole': ['[NH]1C=NC=C1'],
    'imidiz': ['[NH]1C=CC=C1'],
    'pyridine': ['n1cnccc1'],
    'pyrimidine': ['n1ccccc1']
}
#clean up
def derive(ligand, group):
    mol = Chem.MolFromMol2File(f"data/MOL2_files/{ligand}_H.mol2",sanitize=False)
    old_substr = Chem.MolFromSmarts(group)
    if functional_groups_dict[group] in groups_dict:
        replacement_values = groups_dict[functional_groups_dict[group]]
        for a in replacement_values:
            if a != group:
                new_substr = Chem.MolFromSmiles(a)
                rms = AllChem.ReplaceSubstructs(mol, old_substr, new_substr)
                fragments = Chem.GetMolFrags(rms[0])
                for frag in fragments:
                    ind_frag = Chem.MolFragmentToSmiles(rms[0], frag)
                    if (len(ind_frag) > 25) & (ind_frag != "O[H][H]") & (ind_frag not in derivative_smile):
                        derivative_smile.append(ind_frag)
                        temp_mol = Chem.MolFromSmiles(ind_frag)
                        if temp_mol is not None:
                            derivative_mols.append(temp_mol)

for i in ligs:
    mol_groups = []
    mol = Chem.MolFromMol2File(f"data/MOL2_files/{i}_H.mol2", sanitize = False)
    for j in functional_groups:
        k = Chem.MolFromSmarts(j)
        if mol.HasSubstructMatch(k):
            mol_groups.append(1) # 1 corresponds to a functional group being present
            print(f'Atom indices for substructure match (type: {functional_groups_dict[j]}) in ligand {i}: ')
            idxes = mol.GetSubstructMatches(k)
            idxes_list = list(idxes)
            print(idxes)
        else:
            mol_groups.append(0) # 0 corresponds to a functional group being absent
    all_func_groups.append(mol_groups)
    
for location, i in enumerate(ligs):
    matches = []
    for func_loc, j in enumerate(all_func_groups[location]):
        if j == 1:
            matches.append(functional_groups[func_loc]) #list of smiles strings for all functional groups in ligand
        for group in matches:
            derive(i, group)

To get the functional groups in each derivative, a similar method used to determine the functional groups in the original ligand/s is utilized, with a few small changes. The output of this function is a dictionary, where the keys are the indexes of atoms determined to be in a functional group, and the corresponding value is the name of the functional group. Due to keys being unable to be used more than once in a dictionary, atoms that are members of two or more functional groups will only have one of their functional groups listed as the value.

In [ ]:
group_idxes_from_mol(derivative_mols[1])

In order to dock the derivatives, local files in mol2 and pdbqt formats need to be created. As rdkit molecules cannot be converted to these file types directly, derivative molecules must first be converted and saved as a pdb file. The names of each created file are added to its respective list (`filenames`, `filenames_H`, and `filenames_pdbqt`)

In [ ]:
# LEE NOTE FROM LEE: clunky. FIX
derivs = []
ligs_with_derivs = []
fxnal_groups_with_derivs = []

for h, i in enumerate(ligs):
    ligs_with_derivs.append(i)
    fxnal_groups_with_derivs.append(all_func_groups[h])

for mol_num, mol in enumerate(derivative_mols):
    new_ligand = 'derivative_' + str(mol_num)
    pdb_file = 'data/PDB_files/' + str(new_ligand) + '.pdb'
    Chem.MolToPDBFile(mol, pdb_file)
    
    # make mol2 file
    pdb_mol = [m for m in pybel.readfile(filename = pdb_file, format='pdb')][0]
    out = pybel.Outputfile(filename= f"data/MOL2_files/{new_ligand}.mol2", overwrite = True, format='mol2')
    out.write(pdb_mol)
    out.close()
    
    # make mol2 file (including hydrogens)
    mol2_mol = [m for m in pybel.readfile(filename = f"data/MOL2_files/{new_ligand}.mol2", format='mol2')][0]
    mol2_mol.make3D()
    mol2_mol.addh()
    out = pybel.Outputfile(filename= f"data/MOL2_files/{new_ligand}_H.mol2", overwrite = True, format='mol2')
    out.write(mol2_mol)
    out.close()
    
    # make pdbqt file
    out2 = pybel.Outputfile(filename= f"data/PDBQT_files/{new_ligand}.pdbqt", overwrite = True, format='pdbqt')
    out2.write(mol2_mol)
    out2.close()

    # get functional groups of derivatives
    mol_groups = []
    for j in functional_groups:
        k = Chem.MolFromSmarts(j)
        if mol.HasSubstructMatch(k):
            mol_groups.append(1) # 1 corresponds to a functional group being present
        else:
            mol_groups.append(0) # 0 corresponds to a functional group being absent
    fxnal_groups_with_derivs.append(mol_groups)
    
    # append lists with derivatives
    derivs.append(new_ligand)
    ligs_with_derivs.append(new_ligand)
    filenames.append(f"data/MOL2_files/{new_ligand}.mol2")
    filenames_H.append(f"data/MOL2_files/{new_ligand}_H.mol2")
    filenames_pdbqt.append(f"data/PDBQT_files/{new_ligand}.pdbqt")


In [ ]:
Draw.MolsToGridImage(derivative_mols, molsPerRow=5, subImgSize=(300,300))

## Dock and Analyze Derivatives

The derivatives can be docked against the receptor using the same procedure used to dock the original ligands in the previous notebook (`docking-and-prelim-analysis`). The number of derivatives to be docked and the derivatives that will be used depends on the selection/s using the widget above. As the procedure is the same, the step-by-step guide for VINA and SMINA is identical to the one in `docking-and-prelim-analysis`.

Docking every generated derivative would quickly use up a lot of memory. The widget created by the cell below allows for both a single derivative and multiple derivatives to be docked. To select multiple derivatives, hold down the control key (PC) or command key (Mac) while clicking on the names of each derivative you would like to dock. The names of a given derivative corresponds to its location in the grid above (for example, `derivative_0` is at index 0 and is the first derivative shown in the grid)

In [ ]:
style = {'description_width': 'initial'}
select_drop = SelectMultiple(options = derivs, description = 'Select Derivative/s to Dock:', style = style)
select_drop

Ligand docking can either be site-specific or blind. Site-specific docking uses a location of the receptor where we know the ligand binds, and uses the center and size of the ligand as determined in docking_prep. Blind docking attempts to bind the ligand in multiple potential pockets in the protein (determined using fpocket in docking_prep) and requires more computational energy to perform. The option selected in the dropbox below will determine the method used in this notebook.

In [ ]:
style = {'description_width': 'initial'}
select_type = Dropdown(options = ["Site-specific docking","Blind docking"], description = 'Select Docking Type:', style = style)
select_type

### Docking with Vina

Below is a step-by-step (cell-by-cell) guide on how the VINA docking engine is used to generate poses and scores for each pocket and ligand

- Prior to docking, two new folders are created in the data folder to organize the output data (vina_out and vina_out_2).
- Using the information collected in the docking-prep notebook, each pocket's center values and size values are added to their respective lists, which are called pocket_center and pocket_size. In both lists, each instance is a list of the x, y, and z values corresponding to one pocket's data (as a result, pocket_center and pocket_size are nested lists, and the length of both lists is equal to the number of binding pockets)
    - For example, pocket_center may look like this: [[x1, y1, z1][x2, y2, z2][x3, y3, z3]]
- Using the pocket size and center lists and the pdbqt files for the receptor and desired ligand, ligand poses are generated for each binding pocket (the number of poses depends on the value of n_poses, which is set to 5 in this notebook). The amount of computational effort needed to generate the poses for a given pocket and ligand is called the exhaustiveness. As exhaustiveness increases, the more reproducible the results tend to be. While the default value of exhaustiveness is 8, this notebook uses an exhaustiveness of 5 due to memory limitations.
- The results of running the VINA docking engine are stored as pdbqt files and can be located in the vina_out folder. In order to analyze and vizualize the results, the pdbqt files are converted into sdf files using the function pdbqt_to_sdf (created by Angel Ruiz-Moreno), which can be found in the vina_out_2 folder. The names of each file follows the formula of `(ligand name)_vina_pocket_(pocket number).pdbqt` for the pdbqt files and `(ligand name)_pocket_(pocket number)_(name of folder).sdf` for the sdf files.

In [ ]:
from vina import Vina

current_dir = os.getcwd()
vina_out = os.path.join(current_dir, "data", "vina_out")
vina_out_2 = os.path.join(current_dir, "data", "vina_out_2")

pocket_center = []
pocket_size = []

for i in select_drop.value:
    for pocket in prot_pockets.index:
        c_x = prot_pockets.loc[pocket,'center_x']
        c_y = prot_pockets.loc[pocket,'center_y']
        c_z = prot_pockets.loc[pocket,'center_z']
        s_x = prot_pockets.loc[pocket,'size_x']
        s_y = prot_pockets.loc[pocket,'size_y']
        s_z = prot_pockets.loc[pocket,'size_z']
        pocket_center.append([c_x, c_y, c_z])
        pocket_size.append([s_x, s_y, s_z])

In [ ]:
def vina_dock(ligand):
    v = Vina(sf_name='vina')
    v.set_receptor(f'data/PDBQT_files/{pdb_id}_protein.pdbqt')
    v.set_ligand_from_file(f"data/PDBQT_files/{ligand}.pdbqt")
    if select_type == "Blind docking":
        for pock_num, pocket in enumerate(prot_pockets.index):
            v.compute_vina_maps(center = pocket_center[pock_num], box_size = pocket_size[pock_num])
            v.dock(exhaustiveness=5, n_poses=5)
            v.write_poses("data/vina_out/" + str(ligand) + "_vina_pocket_" + str(pocket) + '.pdbqt', n_poses=5, overwrite=True)
    else:
        # need to fix center and size for derivatives
        v.compute_vina_maps(center = center[derivs.index(ligand)], box_size = size[derivs.index(ligand)])
        v.dock(exhaustiveness=5, n_poses=5)
        v.write_poses("data/vina_out/" + str(ligand) + '.pdbqt', n_poses=5, overwrite=True)

In [ ]:
for i in select_drop.value:
    vina_dock(i)

In [ ]:
for i in select_drop.value:
    if select_type == "Blind docking":
        for pocket in prot_pockets.index:
            pdbqt_to_sdf(pdbqt_file=f"data/vina_out/{i}_vina_pocket_{pocket}.pdbqt",output=f"data/vina_out_2/{i}_pocket_{pocket}_vina_out_2.sdf")
    else:
        pdbqt_to_sdf(pdbqt_file=f"data/vina_out/{i}.pdbqt",output=f"data/vina_out_2/{i}_vina_out_2.sdf")

### Docking with Smina

Below is a step-by-step (cell-by-cell) guide on how the SMINA docking engine is used to generate poses and scores for each pocket and ligand
- Prior to docking, two new folders are created in the data folder to organize the output data (smina_out and smina_out_2). The path for the smina docking engine executable is also initialized to allow for the docking engine to be used, as it is a local file.
- Using the the pdbqt file for the receptor, the mol2 file for the desired ligand, and the pocket center/size values from the prot_pockets dataframe, ligand poses are generated for each binding pocket (the number of poses depends on the value of num_modes, which is set to 5 in this notebook). The amount of computational effort needed to generate the poses for a given pocket and ligand is called the exhaustiveness. As exhaustiveness increases, the more reproducible the results tend to be. While the default value of exhaustiveness is 8, this notebook uses an exhaustiveness of 5 due to memory limitations.
- The results of running the SMINA docking engine are stored as sdf files and can be located in the smina_out folder. However, due to the fact that the output files do not have a flag marking it as three dimensional, the sdf files must be read using SDMolSupplier and re-written using SDWriter to avoid excessive errors. The re-written sdf files can be found in the smina_out_2 folder. The names of each file follows the formula of `(ligand name)_pocket_(pocket number)_(name of folder).sdf` for the sdf files.

In [ ]:
# Using SMINA to dock ligand/s in docking boxes based on fpocket's identified pockets
d = 0
for i in select_drop.value: 
    if select_type.value == "Blind docking":
        for pock_num, pocket in enumerate(prot_pockets.index):
            rec = f'data/PDBQT_files/{pdb_id}_protein.pdbqt'
            lig = f'data/MOL2_files/{i}_H.mol2'
            outfile = f'data/smina_out/{i}_pocket_{pocket}_smina_out.sdf'
            ! smina -r {rec} -l {lig} -o {outfile} -center_x {pocket_center[pock_num][0]} -center_y {pocket_center[pock_num][1]} -center_z {pocket_center[pock_num][2]} -size_x {pocket_size[pock_num][0]} -size_y {pocket_size[pock_num][1]} -size_z {pocket_size[pock_num][2]} --exhaustiveness 5 --num_modes 5
    else:
        rec = f'data/PDBQT_files/{pdb_id}_protein.pdbqt'
        lig = f'data/MOL2_files/{i}_H.mol2'
        outfile = f'data/smina_out/{i}_smina_out.sdf'
        ! smina -r {rec} -l {lig} -o {outfile} --autobox_ligand {lig} --autobox_add 5 --exhaustiveness 5 --num_modes 5

In [ ]:
# Rewrite .sdf output files to add 3D tag
# This code will result in warnings. This is normal as long as the warning is
# "Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D."
mols_all = []
for i in select_drop.value:
    mols = []
    if select_type.value == "Blind docking":
        for pocket in prot_pockets.index:
            with Chem.SDMolSupplier(f'data/smina_out/{i}_pocket_{pocket}_smina_out.sdf') as suppl:
                for mol in suppl:
                    if mol is not None:
                        Chem.MolToMolBlock(mol)
                        mols.append(mol)
            with Chem.SDWriter(f"data/smina_out_2/{i}_pocket_{pocket}_smina_out_2.sdf") as w:
                for mol in mols:
                    w.write(mol)
    else:
        with Chem.SDMolSupplier(f'data/smina_out/{i}_smina_out.sdf') as suppl:
            for mol in suppl:
                if mol is not None:
                    Chem.MolToMolBlock(mol)
                    mols.append(mol)
        with Chem.SDWriter(f"data/smina_out_2/{i}_smina_out_2.sdf") as w:
            for mol in mols:
                w.write(mol)

## Analysis of derivative docking output

Now that we have results from molecular docking, we need to make sense of the information. If you were to open the sdf files in a text editor, you would see x, y, and z coordinates for each atom in the ligand, the bond types between atoms in the ligand, and the score of the ligand pose. While useful, this information is difficult to interpret and visualize. To get information regarding the number of interactions, the types of interaction, and the atoms (ligand) and residues (receptor) involved in binding the ligand to the receptor, interaction fingerprints (IFPs) can be generated and viewed using the prolif library, which can be used to identify key atoms in the ligand and key residues in the receptor involved in protein-ligand complex formation.

In [ ]:
style = {'description_width': 'initial'}
select_dock = Dropdown(options = ["smina", "vina"], description = 'Select Docking Engine:', style = style)
select_dock

(get scores for all results)

In [ ]:
all_results = []
scores = [] # get list of scores for each pose
for h, i in enumerate(select_drop.value):
    # initialize list that contains values all poses in all pockets (for 1 ligand at a time)
    nested_results = []
    # append pose data for each pocket to nested_results
    if select_type.value == "Blind docking":
        for pocket in prot_pockets.index:
            results = Chem.SDMolSupplier(f"data/{select_dock.value}_out_2/{i}_pocket_{pocket}_{select_dock.value}_out_2.sdf")
            nested_results.append(results)
    else:
        results = Chem.SDMolSupplier(f"data/{select_dock.value}_out_2/{i}_{select_dock.value}_out_2.sdf")
        nested_results.append(results) 
    # add all values in nested_results to allResults list
    all_results.append(nested_results)
    
# get score values for every pose in allResults
if select_type.value == "Blind docking":
    for linenum, i in enumerate(all_results):
        for num, pocket in enumerate(i):
            for num2, pose in enumerate(pocket):
                if select_dock.value == "smina":
                    scores.append(float(all_results[linenum][num][num2].GetProp('minimizedAffinity')))
                else:
                    scores.append(float(all_results[linenum][num][num2].GetProp('Score')))
else:
    for linenum, i in enumerate(all_results):
        for num, pose in enumerate(i):
            for num2, item in enumerate(pose):
                if select_dock.value == "smina":
                    scores.append(float(all_results[linenum][num][num2].GetProp('minimizedAffinity')))
                else:
                    scores.append(float(all_results[linenum][num][num2].GetProp('Score')))

(Get interactions for derivatives)

In [ ]:
prot_mol = Chem.MolFromPDBFile("data/PDB_files/" + str(pdb_id) + "_protein_H.pdb")
protein_plf = plf.Molecule.from_rdkit(prot_mol)

In [ ]:
all_ligand_plf = []
ligand_plf = []
all_df = []
all_ifps = []
for i in select_drop.value:
    if select_type.value == "Blind docking":
        for pocket in prot_pockets.index:
            lig_suppl = plf.sdf_supplier(f"data/{select_dock.value}_out_2/{i}_pocket_{pocket}_{select_dock.value}_out_2.sdf")
            fp = plf.Fingerprint(count=True)
            fp.run_from_iterable(lig_suppl, protein_plf)
            results_df = fp.to_dataframe()
            all_df.append(results_df)
            for lig in lig_suppl:
                all_ligand_plf.append(lig)
                ifp = fp.generate(lig, protein_plf, metadata = True)
                all_ifps.append(ifp)
    else:
        lig_suppl = plf.sdf_supplier(f"data/{select_dock.value}_out_2/{i}_{select_dock.value}_out_2.sdf")
        fp = plf.Fingerprint(count=True)
        fp.run_from_iterable(lig_suppl, protein_plf)
        results_df = fp.to_dataframe()
        all_df.append(results_df)
        for lig in lig_suppl:
            all_ligand_plf.append(lig)
            ifp = fp.generate(lig, protein_plf, metadata = True)
            all_ifps.append(ifp)

In [ ]:
# OLD
if select_type.value == "Blind docking":
    df = pd.concat([d for d in all_df], axis=0, ignore_index=False, sort=False, keys = prot_pockets.index).reset_index()
else:
    df = pd.concat([d for d in all_df], axis=0, ignore_index=False, sort=False).reset_index()
df.insert(2, "Score", pd.Series(scores))
df = df.fillna(0)

In [ ]:
# prot pockets not listed - try to fix?
df = pd.concat([d for d in all_df], axis=0, ignore_index=False, sort=False).reset_index()
df.insert(1, "Score", pd.Series(scores))
df = df.fillna(0)

While the dataframe generated using the prolif library has a lot of useful information, we are also going to add the distance between interacting ligand and protein atoms, the indexes of both the ligand and protein atoms involved in the interaction, and the functional group the ligand's atom is a member of if applicable.

In [ ]:
# fix for site specific
if select_type.value == "Blind docking":
    #df2 = df[["cav_id", "Frame", "Score"]].copy()
    df2 = df[["Frame", "Score", "UNL1"]].copy()
else:
    df2 = df[["Frame", "Score", "UNL1"]].copy()

In [ ]:
largest_array_column = {}
for col_num, column in enumerate(df):
    largest_array = 0
    if col_num > 1:
        for row in df[column]:
            if int(row) > largest_array:
                largest_array = int(row)
        largest_array_column[column] = largest_array

In [ ]:
%%capture
# create new columns for functional group, residue type, distance, and index information
# LEE NOTE TO LEE: fix how columns are added. make pretty
col_names_list = []
residues = []
interactions = []
total_counter = 0
for key in all_ifps:
    for key_new in key:
        for key_2 in key[key_new]:
            print(key_new)
            residues.append(str(key_new[1]))
            interactions.append(str(key_2))
            lig_name = str(key_new[0])
            res_name = str(key_new[1])
            column_name = (lig_name, res_name, key_2)
            if column_name not in col_names_list:
                df2[column_name] = df[column_name]
                number_of_ints = int(largest_array_column[column_name])
                counter_ind = 0
                while counter_ind < number_of_ints:
                    df2[(lig_name, res_name, f"Functional group involved ({key_2}){counter_ind}")] = pd.Series([0] * df.shape[0])
                    df2[(lig_name, res_name, f"Residue type({key_2}){counter_ind}")] = pd.Series([0] * df.shape[0])
                    df2[(lig_name, res_name, f"Distance ({key_2}){counter_ind}")] = pd.Series([0] * df.shape[0])
                    df2[(lig_name, res_name, f"Index 1 (Ligand) ({key_2}){counter_ind}")] = pd.Series([0] * df.shape[0])
                    df2[(lig_name, res_name, f"Index 2 (Ligand) ({key_2}){counter_ind}")] = pd.Series([0] * df.shape[0])
                    df2[(lig_name, res_name, f"Index 3 (Protein) ({key_2}){counter_ind}")] = pd.Series([0] * df.shape[0])
                    df2[(lig_name, res_name, f"Index 4 (Protein) ({key_2}){counter_ind}")] = pd.Series([0] * df.shape[0])
                    counter_ind += 1
            total_counter += 1

In [ ]:
# create amino acid residue and functional group dictionaries
all_func_groups, type_dict, functional_groups, functional_groups_dict, groups_to_numbers, groups_dict = get_vars()

# find atom indices for ligand and protein, functional groups involved (ligand), residue type (protein), and
# distance between ligand and protein in interaction
data_dict = {}
total_counter = 0
for number, key in enumerate(all_ifps):
    for key_new in key:
        for key_2 in key[key_new]:
            lig_name = str(key_new[0])
            res_name = str(key_new[1])
            column_name = (lig_name, res_name, key_2)
            get_pose = df2["Frame"][number]
            x = key[key_new]
            y = x[key_2]
            df_groups = [0] * largest_array_column[column_name]
            df_residue = [0] * largest_array_column[column_name]
            df_distance = [0] * largest_array_column[column_name]
            df_ind_1 = [0] * largest_array_column[column_name]
            df_ind_2 = [0] * largest_array_column[column_name]
            df_ind_3 = [0] * largest_array_column[column_name]
            df_ind_4 = [0] * largest_array_column[column_name]
            for inst_num, instance in enumerate(y):
                distance = instance["distance"]
                df_distance[inst_num] = distance
                found_res = res_name[:3]
                df_residue[inst_num] = (type_dict[found_res])
                parent_index = instance["parent_indices"]
                if len(parent_index["ligand"]) == 2:
                    df_ind_1[inst_num] = parent_index["ligand"][0]
                    df_ind_2[inst_num] = parent_index["ligand"][1]
                else:
                    df_ind_1[inst_num] = parent_index["ligand"][0]
                    df_ind_2[inst_num] = 0
                if len(parent_index["protein"]) == 2:
                    df_ind_3[inst_num] = parent_index["protein"][0]
                    df_ind_4[inst_num] = parent_index["protein"][1]
                else:
                    df_ind_3[inst_num] = parent_index["protein"][0]
                    df_ind_4[inst_num] = 0
                current = all_ligand_plf[number]
                group_ints = group_idxes_from_mol(current)
                for value in group_ints.keys():
                    if len(parent_index["ligand"]) == 2:
                        if value == parent_index["ligand"][0] | value == parent_index["ligand"][1]:
                            df_groups[inst_num] = int(groups_to_numbers[group_ints[value]])
                    else:
                        if value == parent_index["ligand"][0]:
                            df_groups[inst_num] = int(groups_to_numbers[group_ints[value]])
            number_of_ints = int(largest_array_column[column_name])
            counter_ind = 0
            while counter_ind < number_of_ints:
                df2.at[number, (lig_name, res_name, f"Functional group involved ({key_2}){counter_ind}")] = df_groups[counter_ind]
                df2.at[number, (lig_name, res_name, f"Residue type({key_2}){counter_ind}")] = df_residue[counter_ind]
                df2.at[number, (lig_name, res_name, f"Distance ({key_2}){counter_ind}")] = df_distance[counter_ind]
                df2.at[number, (lig_name, res_name, f"Index 1 (Ligand) ({key_2}){counter_ind}")] = df_ind_1[counter_ind]
                df2.at[number, (lig_name, res_name, f"Index 2 (Ligand) ({key_2}){counter_ind}")] = df_ind_2[counter_ind]
                df2.at[number, (lig_name, res_name, f"Index 3 (Protein) ({key_2}){counter_ind}")] = df_ind_3[counter_ind]
                df2.at[number, (lig_name, res_name, f"Index 4 (Protein) ({key_2}){counter_ind}")] = df_ind_4[counter_ind]
                counter_ind += 1
            total_counter += 1

In [ ]:
df2 = df2.convert_dtypes()
df2.to_csv('data/deriv_information.csv', index = False)
df2

### Title here

Using an IFP, the interactions between the ligand and the receptor can be visualized using prolif's Complex3D submodule. Only one pose and its interactions can be viewed at a time.

In [ ]:
# display interactions. select which one to view using dropdown
pose_pock_select = []
a = 0
while a < int(df.shape[0]):
    pose_pock_select.append(a + 1)
    a += 1
style = {'description_width': 'initial'}
select_pose = Dropdown(options = pose_pock_select, description = 'Select Pose to View:', style = style)
select_pose

In [ ]:
comp = Complex3D(all_ifps[select_pose.value], all_ligand_plf[select_pose.value], protein_plf)
comp.display()

## Save results for further use

In [ ]:
deriv_smiles_data = pd.DataFrame({"filename_hydrogens": filenames_H,
                                   "smiles": derivative_smile})
deriv_smiles_data.to_csv('data/deriv_smiles_data.csv', index = False)